In [8]:
import datetime
import os
import pathlib
import shutil
import subprocess
import sys
import typing as t

In [2]:
os.environ['CPATH']

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/papi-5.6.0-5p5d3qi2imn6m3szyl47gj73dpixhxpf/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/amrex-18.08.1-crdvjfdihxbjlam4swf2jzopgjv3dopv/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hypre-2.14.0-vea6lmriirgjzt4o6j4zxw2xbunynfs6/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/openblas-0.3.2-femnsy3tmxlxehbgrk4yd4gmig2zjsle/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hdf5-1.8.20-p357p5eclyvy4vpqaownurdkn3mxv56v/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/zlib-1.2.11-z7cqiqbuz4k4x4ideyvz3syxata5g5gd/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/numactl-2.0.11-nawuf6bcuc

In [6]:
shutil.which('hpcrun')

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/bin/hpcrun'

In [19]:
try:
    _HERE = pathlib.Path(__file__).parent.resolve()
except NameError:
    _HERE = pathlib.Path(os.getcwd()).resolve()

In [13]:
def hpctoolkit_profile(executable, results_path, sample_size: int):
    hpcrun = shutil.which('hpcrun')
    # results_path = "$HERE/results/profile_${NOW_DATE}_${app}_${experiment}"
    hpcrun_command = '{} -o "{}" {}'.format(hpcrun, results_path, executable)
    for i in range(sample_size):
        subprocess.run(hpcrun_command, shell=True)

In [14]:
def hpctoolkit_summarize(executable, results_path, source_path):
    hpcstruct = shutil.which('hpcstruct')
    hpcprof = shutil.which('hpcprof')
    # results_path = "$HERE/results/profile_${NOW_DATE}_${app}_${experiment}"
    
    struct_path = results_path.join(executable.name + '.hpcstruct')
    
    hpcstruct_command = '{} -I {} -o {} {}'.format(
        hpcstruct, source_path, struct_path, executable)
    subprocess.run(hpcstruct_command, shell=True)
    
    # hpcprof -I "${source}" "${results_path}" \
    #   -S "${results_path}/flash4.hpcstruct" \
    #   -M stats \
    #   -o "${results_path}_db"
    hpcprof_command = '{} -I {} {} -S {} -M stats -o {}'.format(
        hpcprof, source_path, results_path, struct_path, str(results_path) + '_db')
    subprocess.run(hpcprof_command, shell=True)

In [20]:
def profile_flash(app_name, experiment, sample_size: int):
    _NOW = datetime.datetime.now()
    executable = './flash'
    results_path = pathlib.Path(_HERE, 'results', 'profile_{}_{}_{}'.format(
        _NOW.strftime('%Y%m%d-%H%M%S'), app_name, experiment))
    source_path = pathlib.Path(_HERE, '')
    
    hpctoolkit_profile(executable, results_path, sample_size)
    hpctoolkit_summarize(executable, results_path, source_path)

In [ ]:
def _run_and_check(test_name: str, cmd: str, wd: pathlib.Path, log_filename_prefix):
    cmd_result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                shell=True, cwd=str(wd))
    cmd_msg = None
    if cmd_result.returncode != 0:
        log_dir = pathlib.Path(
            _HERE, 'results', '{}_{}'.format(_NOW.strftime('%Y%m%d-%H%M%S'), test_name))
        log_dir.mkdir(parents=True, exist_ok=True)
        cmd_stdout = cmd_result.stdout.decode()
        with open(str(pathlib.Path(log_dir, '{}_stdout.log'.format(log_filename_prefix))),
                  'w') as cmd_stdout_file:
            cmd_stdout_file.write(cmd_stdout)
        cmd_stderr = cmd_result.stderr.decode()
        with open(str(pathlib.Path(log_dir, '{}_stderr.log'.format(log_filename_prefix))),
                  'w') as cmd_stderr_file:
            cmd_stderr_file.write(cmd_stderr)
        cmd_msg = '"{}" failed, returncode={}, logs were written to "{}"' \
            ' and last 50 lines of stderr follow:\n{}'.format(
                cmd, cmd_result.returncode, log_dir,
                ''.join(cmd_stderr.splitlines(keepends=True)[-50:]))
    assert cmd_result.returncode == 0, cmd_msg

In [ ]:
def setup_flash(test_name, experiment, setup_dir):
    _run_and_check(test_name, './setup -site spack ' + expermient, setup_dir, 'setup')

In [ ]:
def build_flash(test_name):
    _run_and_check(test_name, 'make')

In [ ]:
app_name = 'flash-subset'
experiment 'Sedov -auto -2d +Mode1'
branch = 'transpyle_experiments_outlined'

# checkout right branch
setup_flash()
profile_flash()